# Travelling Salesman Problem <br /> Graph2Graph approach

In [63]:

import pickle

import torch
import torch.nn as nn
import torch.functional as F
from torch.nn import Linear, ReLU
from torch_geometric.nn import Sequential, SAGEConv
from torch_geometric.utils import softmax

In [64]:
with open('data/dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
print("dataset size: ", len(dataset))

dataset size:  10


In [65]:
data, opt_route, opt_length = dataset[0]

In [66]:
print(data.edge_index[0,0], "->", data.edge_index[1,0])

tensor(0) -> tensor(6)


In [71]:
class GNNEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, drop_rate = 0.1):
        super().__init__()
        self.dropout = drop_rate
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)

    def forward(self, data):
        # Define a 2-layer GNN computation graph.
        # Use a *single* `ReLU` non-linearity in-between.
        x = self.conv1(data.x, data.edge_index)
        x = F.Relu(x)
        x = F.dropout(x, p=self.dropout)

        x = self.conv2(x, data.adj_t)
        x = F.Relu(x)
        x = F.dropout(x, p=self.dropout)

        x = self.conv3(x, data.adj_t)
        x = F.Relu(x)
        x = F.dropout(x, p=self.dropout)
        return x

class DotDecoder(nn.Module):
    def forward(self, x):
        logit = torch.matmul(x, x)
        return logit

class Graph2Graph(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.encoder = GNNEncoder(input_dim, hidden_dim)
        self.decoder = DotDecoder()
        self.softmax = softmax

    def forward(self, data):
        z = self.encoder(data)
        pi = self.decoder(z)
        return pi

model = Graph2Graph(input_dim=10, hidden_dim=64)

print(model)

Graph2Graph(
  (encoder): GNNEncoder(
    (conv1): SAGEConv(10, 64, aggr=mean)
    (conv2): SAGEConv(64, 64, aggr=mean)
  )
  (decoder): DotDecoder()
)


In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 6):
    total_loss = total_examples = 0
    optimizer.zero_grad()

    # TODO: Move `sampled_data` to the respective `device`
    data.to(device)
    # TODO: Run `forward` pass of the model
    x = model.forward(data)
    # TODO: Apply binary cross entropy via
    print(x)
    # `F.binary_cross_entropy_with_logits(pred, ground_truth)`
    raise NotImplementedError

    loss.backward()
    optimizer.step()
    total_loss += float(loss) * pred.numel()
    total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cpu'


RuntimeError: mat1 and mat2 must have the same dtype